# NASDAQ Technical Analysis & Trends

get nasdaque historical prices<br> 
store it in a database as we plan to add new features<br> 
retrieve from database and create new dataset<BR>
define new features such as day & month that will be used for seasonality <br> 
introduce moving averages <br> 
add some visizuliations<br> 
use Github<br> 




In [2]:
import psycopg2
from psycopg2 import sql
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt

### Source data from yahoo finance

In [3]:
# Nasadq composite ticker symbol
ticker = '^ixic'

# Fetch data for a ticker
df = yf.download(ticker, start='2024-01-01', end='2024-09-18')

[*********************100%%**********************]  1 of 1 completed


In [4]:
df.reset_index(inplace=True)
df['Ticker'] = ticker
df.head(1)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker
0,2024-01-02,14873.700195,14887.799805,14682.379883,14765.94043,14765.94043,5791170000,^ixic


### Connecting to PostgresSQL and loading into a dataframe called df_sql

In [5]:
# psycopg2 is a PostgreSQL database adapter for Python. It allows Python programs to interact with PostgreSQL databases.
# creating a varible to connect with the database.

conn = psycopg2.connect(
    dbname="Stocks",
    user="postgres",
    password="Araofooaca47@41",
    host="localhost",
    port="5432"
)
# Enable autocommit
conn.autocommit = True
cur = conn.cursor()    # The cursor allows python to interact with Postgres SQL database.

In [6]:
# Query the data from the "stocks" table and load it into a DataFrame
query = 'SELECT * FROM stock_symbol'
df_sql = pd.read_sql(query, conn)



C:\Users\rinde\AppData\Local\Temp\ipykernel_2220\3161469733.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, conn)


In [7]:
df_sql.head(1)

,id,open,high,low,close,adj_close,volume,ticker,date,day,month
0,404,14873.7,14887.8,14682.38,14765.94,14765.94,5.791170e+09,^ixic,2024-01-02,Tuesday,January


In [8]:
# cur.execute('''
#     SELECT company_info.company, company_info.ticker, stock_symbol.high, stock_symbol.low, stock_symbol.volume
#     FROM company_info
#     INNER JOIN stock_symbol
#     ON company_info.ticker = stock_symbol.ticker;
#  ''')

# # Fetch all the results
# rows = cur.fetchall()

# # Print the results
# for i in rows:
#     print(i)

In [9]:
# df.tail()

In [10]:
# for _, r in df.iterrows():
#     insert_query = '''
#         INSERT INTO stock_symbol (open, high, low, close, adj_close, volume, ticker, date)
#         VALUES (%s, %s, %s, %s, %s, %s, %s,%s);
#         '''
#     cur.execute(insert_query, (r['Open'], r['High'], r['Low'], r['Close'], r['Adj Close'], r['Volume'], r['Ticker'], r['Date']))

# conn.commit()


In [11]:
# df = df.reset_index()

### Adding two new features called day and month derived from Date column

In [12]:
# Changing Date colum to a date type so we can derive into day and month
df['Date'] = pd.to_datetime(df['Date'])

# Adding a day column and month column to the dataset
df['Day'] = df['Date'].dt.day_name()
df['Month'] = df['Date'].dt.month_name()


In [13]:
df.tail(1)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Day,Month
178,2024-09-17,17707.019531,17781.740234,17544.640625,17628.060547,17628.060547,4959520000,^ixic,Tuesday,September


### Adding new features columns, day and month, into the postgres table stock_symbol.


In [14]:
columns_query = '''
    SELECT column_name
    FROM information_schema.columns
    WHERE table_name = 'stock_symbol';
'''
cur.execute(columns_query)
columns = cur.fetchall()

In [15]:
print(columns)

[('id',), ('open',), ('high',), ('low',), ('close',), ('adj_close',), ('volume',), ('ticker',), ('date',), ('day',), ('month',)]


In [16]:
if ('day',) not in columns:
    cur.execute(
        '''
        ALTER TABLE "stock_symbol"
        ADD COLUMN "day" TEXT;
        '''
        )

In [17]:
if('month',) not in columns:
    cur.execute(
     '''
     ALTER TABLE "stock_symbol"
     ADD COLUMN "month" TEXT;
    '''
    )

### Adding data from df to postgresSQL along with new features

In [18]:
# Deleting all data from table so we can upload fresh data/
cur.execute('''DELETE FROM stock_symbol''')

In [19]:
# Iterating through the rows and inserting into table
for _, r in df.iterrows():
    insert_query = '''
        INSERT INTO stock_symbol (open, high, low, close, adj_close, volume, ticker, date, day, month)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        '''
    cur.execute(insert_query, (r['Open'], r['High'], r['Low'], r['Close'], r['Adj Close'], r['Volume'], r['Ticker'], r['Date'],  r['Day'], r['Month']))

conn.commit()

In [20]:
# Query the data from the "stocks" table and load it into a DataFrame
query = 'SELECT * FROM stock_symbol'
df_sql = pd.read_sql(query, conn)
df_sql.tail(1)

C:\Users\rinde\AppData\Local\Temp\ipykernel_2220\2424629201.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, conn)


,id,open,high,low,close,adj_close,volume,ticker,date,day,month
178,761,17707.02,17781.74,17544.64,17628.06,17628.06,4.959520e+09,^ixic,2024-09-17,Tuesday,September


### Seasonality based on day 

In [21]:
# Calculate up and down days


In [22]:
# Add new feature to our database for up_down_day
up_day = df_sql['close'] > df_sql['open']
down_day = df_sql['open'] > df_sql['close']
unchanged = df_sql['open'] = df_sql['close']

In [23]:
# Add to our database

In [24]:
cur.close()
conn.close()

git checkout staging => switch between branches.<BR>
git fetch origin => update your local repository with the latest changes from the remote branch<BR>
git pull origin staging => Merge the remote staging branch into your local staging branch<BR>
git add . => add all changes in the current directory<BR>